# Цель

Практический проект по написанию собственного парсера.
Цель: В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-кода и собрать себе свой собственный датасет.
По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).

Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.

В качестве потенциальных целей для парсинга можно взять какие-нибудь блоги, выкачать оттуда публикации, авторов, число просмотром и комментариев. Можно посмотреть новостные ресурсы и скачать новостные статьи с их мета-информацией. Или любой другой интересный вам сайт.

Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла) 

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import re

from tqdm import tqdm_notebook

# Проектирование

В данном учебном проекте разработаем парсер сайта новостей `coindesk.com` по тематике Bitcoin:

Из множества разделов сайта остановимся на разделе Market: https://www.coindesk.com/category/markets-news/markets-markets-news

Основным объектом в для парсера будет являться новость, новость содержит следующие поля:
- заголовок(title)
- краткое содержание(anonce)
- текст новости(text)
- теги(tags)
- время публикации(create_time)
- автор(author)
- ссылка на новость(article_link)

Создадим класс `Article`, в котором будем хранить информацию об объекте:

In [2]:
class Article:
    def __init__(self, title, anonce, text, tags, create_time, author, article_link):
        self._title         = title
        self._anonce        = anonce
        self._text          = text
        self._tags          = tags
        self._create_time   = create_time
        self._author        = author
        self._article_link  = article_link

        self._attributes    = ['title','anonce','text','tags','create_time','author','article_link']
        
    @property
    def attributes(self): 
        return self.attributes
        
    @property
    def title(self): 
        return self._title

    @property
    def anonce(self):
        return self._anonce

    @property
    def text(self):
        return self._text

    @property    
    def tags(self):
        return self._tags

    @property
    def create_time(self):
        return self._create_time

    @property
    def author(self):
        return self._author

    @property    
    def article_link(self):
        return self._article_link

    def __str__(self):
        return str(self.todict())
    
    def todict(self):
        return {
            'title':self._title,
            'anonce':self._anonce,
            'text':self._text,
            'tags':self._tags,
            'create_time':self._create_time,
            'author':self._author,
            'article_link':self._article_link
        }
    
    def __repr__(self):
        return self.__str__()

Извелеченные объекты будет складывать в специальное хранилище. Для хранилища будем учитывать, что оно может иметь множества реализаций и основной интерфейс работы должен быть одинаков.

Из основного интерфейса абстрактного класса `Store` (не совсем, конечно, абстрактного) описаны основные методы:
- Store.push() - складываем новость класса `Article`
- Store.push_list() - складываем список новостей класса `Article`
- Store.commit() - Фиксируем(сохраняем) помещенный объем

Также создадим двух наследников от класса `Store`: `ListStore`, `FileStore`.

- `ListStore` - Сохраняем объекты `Article` в список (при этом не контроллируем использование памяти)
- `FileStore` - Сохраняем объекты `Article` в csv-файл (контроллируем память и не зависим от объема извлекаемой информации на сайте)

In [3]:
class Store:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def push(self):
        pass
    
    def push_list(self):
        pass
    
    def commit(self):
        pass

class ListStore(Store):
    def __init__(self):
        self._store = []

    def push(self,obj):
        self._store.append(obj)
        
    def push_list(self, list_obj):
        for o in list_obj:
            self.push(o)
        
    def pull(self,index = 0):
        return self._store[index]

    def __str__(self):
        return str(self._store)

    def __len__(self):
        return len(self._store)
    
    def __repr__(self):
        return self.__str__()
    
    def tolist(self):
        # pd.DataFrame([ls.pull(0).todict()])
        store_dict_list = []
        store_len = len(self._store)
        for i in range(0,store_len):
            store_dict_list.append(self.pull(i).todict())
        return store_dict_list

import csv

class FileStore(Store):
    def __init__(self, filename):
        self._file = open(filename, mode='w')
        
        fieldnames = ['title','anonce','text','tags','create_time','author','article_link']
        self._store = csv.DictWriter(self._file, fieldnames=fieldnames)
        
        self._store.writeheader()

    def push(self,obj):
        self._store.writerow(obj.todict())
        
    def push_list(self, list_obj):
        for o in list_obj:
            self.push(o)
            
    def commit(self):
        self._file.close()

Цель разрабатываемой архитектуры не зависеть от хранилища объектов и типов источников(сайтов). Таким образом для данного раздела сайта создадим интерфейс. Так как адрес новости в конце ссылки содержит перечесляемое целое число - идентификатор, отвечающее за страницу с новостями, задача интерфейса вернуть все новости в виде объектов `Article` по идентификатору.

In [5]:
import time, datetime

class CoindeskComInterface:
    def __init__(self,main_page_link, object_type, timeout):
        self._main_page_link = main_page_link
        self._object_type = object_type
        self._timeout = int(timeout)
    
    @property
    def timeout(self):
        return self._timeout
    
    def _load_page_with_id(self, page_id):
        return self._load_page(link_page=self._main_page_link+str(page_id))
    
    def _load_page(self, link_page):
        return requests.get(link_page)
    
    def _get_objects(self, beautiful_page, timeout):
        obj = self._object_type
        obj_list = []
        
        for block in beautiful_page.findAll('a', attrs = {'class':"stream-article"}):
            create_time = block.find('time')['datetime']
            author = block.find('div', attrs = {'class':'time'}).text.split('|')[1].strip()
            anonce = block.find('p').text
            title = block['title']
            link = block['href']
            
            
            print(
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'Load news page',
                link
            )
            
            response = self._load_page(link)
            processed_page_article = response.text.replace('\\','')
            processed_page_article = response.text.replace('\n','')
            beautiful_page_article = BeautifulSoup(processed_page_article, 'lxml')

            text = beautiful_page_article.find('div', attrs={'class':'entry-content'}).text.strip()
            tags = list(map(lambda c: c['content'], beautiful_page_article.findAll('meta', attrs={'property':'article:tag'})))
            tags = ','.join(tags)
            
            o = obj(
                title=title, 
                anonce=anonce, 
                text=text, 
                tags=tags, 
                create_time=create_time, 
                author=author, 
                article_link=link
            )
            
            obj_list.append(o)
            
            time.sleep(timeout)
        
        return obj_list
    
    def get_objects_last_page(self, timeout=0):
        return self.get_objects_page_by_id(0,timeout)
    
    def get_objects_page_by_id(self, page_id, timeout=0):
        response = self._load_page_with_id(page_id=page_id)
        
        if response.ok not in [True]:
            print('Response status:',response.status_code,'link',self._main_page_link+str(page_id))
            return None
            
        print(
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'Load main page',
            self._main_page_link+str(page_id)
        )
        
        try:
            search_result = re.search(r'stream":"(.*)"\}', response.text)
            processed_page = search_result.group(1)
            processed_page = processed_page.replace('\\n','')
            processed_page = processed_page.replace('\\','')

            beautiful_page = BeautifulSoup(processed_page, 'lxml')

            objects = self._get_objects(beautiful_page, timeout)
        except Exception as e:
            print(
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'Data parse error'
            )
            print(e)
            return None
        
        return objects

Сам перебор новостй будем производить в классе `NewsParser`. Его задача получить на вход начальную ссылку, интерфейс и хранилище, итеративным путем перебрать все страницы с новостями и сложить их в хранилище. Перебор будет происходить через указанный на входе итератор, который может быть бесконечным. Для выхода из бесконечно перебора используется порог пустых возвращаемых объектов, когда заданное количество раз интерфейс не смог предоставить список новостей.

In [4]:
class NewsParser:
    def __init__(self, interface, store, timeout, threshold):
        self._interface = interface
        self._store = store
        self._timeout = timeout
        self._threshold = threshold
        self._counter = 0
    
    def run_parser(self, iterator):
        try:
            for i in iterator:
                objs = self._interface.get_objects_page_by_id(i)
                
                if not (objs is None):
                    self._store.push_list(objs)
                else:
                    self._counter += 1
                
                if self._counter >= self._threshold:
                    print(
                        datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Exit, counter =',
                        self._counter,
                        ', threshold =',
                        self._threshold
                    )

                    break
        finally:
            print('file slosed')
            self._store.commit()

# Запуск парсера

Ниже создадим экземпляр интерфейса, файлового хранилища и парсера. На вход парсеру, кроме интерфейса и хранилища, также передаем таймаут в 2 секунды между запросами и порог. 

In [ ]:
from itertools import count

cci = CoindeskComInterface(
    main_page_link='https://www.coindesk.com/wp-json/v1/category/7279/', 
    object_type=Article,
    timeout=2
)

fss = FileStore('news_dataset.csv')

tt = NewsParser(interface=cci, store=fss, timeout=2, threshold=20)
tt.run_parser(count(28))

Данный парсер собрал датасет и сохранил в csv-файл.

Считаем файл через pandas:

In [19]:
df = pd.read_csv('news_dataset.csv')
df

,title,anonce,text,tags,create_time,author,article_link
0,U.K. Exchange Coinfloor Is Getting Paid to Hel...,Coinfloor will collect a fee for referring rep...,"Coinfloor, the U.K.’s longest-running crypto e...","Coinfloor,SEPA,Brexit,Enumis,BACs,CHAPs",2019-05-29T08:00:32+00:00,Ian Allison,https://www.coindesk.com/u-k-exchange-coinfloo...
1,Bitcoin Price Raises Bull Flag in Preparation ...,Bitcoin has formed a technical pattern called ...,ViewBitcoin has formed a bull flag pattern on ...,"Prices,Bitcoin,Markets",2019-05-28T11:00:01+00:00,Omkar Godbole,https://www.coindesk.com/bitcoin-price-raises-...
2,Golden Crossover: XRP Heads for Bullish Chart ...,"XRP is teasing a long-term bullish reversal, w...","XRP is teasing a long-term bullish reversal, w...","Prices,XRP,Markets",2019-05-27T15:30:30+00:00,Omkar Godbole,https://www.coindesk.com/golden-crossover-xrp-...
3,Yahoo Japan-Backed Crypto Exchange Taotao Laun...,A new crypto exchange platform in which Yahoo ...,A new crypto exchange platform in which Yahoo ...,"Exchanges,Japan,Yahoo,Markets,Taotao",2019-05-27T14:20:38+00:00,Daniel Palmer,https://www.coindesk.com/yahoo-japan-backed-cr...
4,"Bitcoin Price Backs Off 12 Month Highs, But Bi...",Bitcoin has retreated from 12-month highs over...,ViewBitcoin has retraced slightly from the one...,"Prices,Bitcoin,Markets",2019-05-27T11:00:26+00:00,Omkar Godbole,https://www.coindesk.com/bitcoin-price-backs-o...
...,...,...,...,...,...,...,...
3640,Argentina trades $50k of bitcoins,"A record $50,000 of bitcoins were traded in Ar...","A record $50,000 of bitcoins were traded in Ar...",Argentina,2013-04-22T14:34:16+00:00,Doug Watt,https://www.coindesk.com/bitcoin-demand-surges...
3641,$500k in funding paves way for bitcoin trading,New York City-based bitcoin startup Coinsetter...,New York City-based bitcoin startup Coinsetter...,NaN,2013-04-17T13:58:05+00:00,Dan Ilett,https://www.coindesk.com/500k-in-funding-paves...
3642,Meet the money behind bitcoin competitor OpenCoin,"At first glance, especially to a digital curre...","At first glance, especially to a digital curre...",Opencoin,2013-04-16T04:54:28+00:00,Dan Ilett,https://www.coindesk.com/meet-the-money-behind...
3643,OpenCoin aims to build a better digital currency,Digital currency promises to solve some proble...,Digital currency promises to solve some proble...,"Opencoin,Ripple,Ripple Labs",2013-04-16T02:53:11+00:00,Dan Ilett,https://www.coindesk.com/opencoin-aims-to-buil...


Посмотрим на получившийся размер:

In [20]:
df.shape

(3645, 7)

Мы собрали 3645 новостей!

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3645 entries, 0 to 3644
Data columns (total 7 columns):
title           3645 non-null object
anonce          3645 non-null object
text            3645 non-null object
tags            3599 non-null object
create_time     3645 non-null object
author          3645 non-null object
article_link    3645 non-null object
dtypes: object(7)
memory usage: 199.5+ KB


# Выводы

1. Был разработан парсер для парсинга сайта новостей
1. По результатам работы парсера был успешно собран датасет